In [ ]:
import os, cv2, keras
import pandas as pd

In [ ]:
# define paths to work with
path = os.getcwd()
abspath = os.path.abspath(os.path.join(path, os.pardir))

In [ ]:
# version for distinguishing models, data, etc
version = "v1"

In [ ]:
# names for folders for the three different models
tf_mm = "tflite_mm"
od_mask = "objdet_mask"
c_class = "celeb_classifier"

In [ ]:
# download images per celebrity (max 477)
results = 5;

In [ ]:
# image resize for training
img_size = 224

In [ ]:
# backups subfolder
backupsPath = f'{abspath}/backups/'
if not os.path.exists(backupsPath):
    os.mkdir(backupsPath)
    
backupVersion = backupsPath + "/" + version
if not os.path.exists(backupVersion):
    os.mkdir(backupVersion)

In [ ]:
# datasets subfolder
datasetsPath = f'{abspath}/datasets'
if not os.path.exists(datasetsPath):
    os.mkdir(datasetsPath)
    
datasetsVersion = datasetsPath + "/" + version
if not os.path.exists(datasetsVersion):
    os.mkdir(datasetsVersion)
    
datasetsTf = datasetsVersion + "/" + tf_mm
datasetsMask = datasetsVersion + "/" + od_mask
datasetsCeleb = datasetsVersion + "/" + c_class

if not os.path.exists(datasetsTf):
    os.mkdir(datasetsTf)
    
tfTrain = datasetsTf + "/train"
tfVal = datasetsTf + "/val"
tfTest = datasetsTf + "/test"

if not os.path.exists(tfTrain):
    os.mkdir(tfTrain)
if not os.path.exists(tfVal):
    os.mkdir(tfVal)
if not os.path.exists(tfTest):
    os.mkdir(tfTest)
    
if not os.path.exists(datasetsMask):
    os.mkdir(datasetsMask)
    
maskMasks = datasetsMask + "/masks"
maskCaffe = datasetsMask + "/caffe"

if not os.path.exists(maskMasks):
    os.mkdir(maskMasks)

if not os.path.exists(maskCaffe):
    os.mkdir(maskCaffe)
    
if not os.path.exists(datasetsCeleb):
    os.mkdir(datasetsCeleb)
    
celebTrain = datasetsCeleb + "/train"
celebTest = datasetsCeleb + "/test"

if not os.path.exists(celebTrain):
    os.mkdir(celebTrain)

if not os.path.exists(celebTest):
    os.mkdir(celebTest)

In [ ]:
# images subfolder
imagesPath = f'{abspath}/images/'
if not os.path.exists(imagesPath):
    os.mkdir(imagesPath)
    
imagesVersion = imagesPath + "/" + version
if not os.path.exists(imagesVersion):
    os.mkdir(imagesVersion)
    
imagesMasks = imagesPath + "/" + "masks"
if not os.path.exists(imagesMasks):
    os.mkdir(imagesMasks)
    
imagesUnknown = imagesPath + "/" + "unknown"
if not os.path.exists(imagesUnknown):
    os.mkdir(imagesUnknown)
    
unknownOriginal = imagesUnknown + "/" + "original"
if not os.path.exists(unknownOriginal):
    os.mkdir(unknownOriginal)
unknownCropped = imagesUnknown + "/" + "crop"
if not os.path.exists(unknownCropped):
    os.mkdir(unknownCropped)
    
imageFolderList = os.listdir(imagesVersion)

In [ ]:
# models subfolder
modelsPath = f'{abspath}/models/'
if not os.path.exists(modelsPath):
    os.mkdir(modelsPath)
    
modelsVersion = modelsPath + "/" + version
if not os.path.exists(modelsVersion):
    os.mkdir(modelsVersion)
    
modelsPretrained = modelsPath + "/" + "pretrained"
if not os.path.exists(modelsPretrained):
    os.mkdir(modelsPretrained)
    
modelsVTf = modelsVersion + "/" + tf_mm
modelsVMask = modelsVersion + "/" + od_mask
modelsVCeleb = modelsVersion + "/" + c_class
if not os.path.exists(modelsVTf):
    os.mkdir(modelsVTf)
if not os.path.exists(modelsVMask):
    os.mkdir(modelsVMask)
if not os.path.exists(modelsVCeleb):
    os.mkdir(modelsVCeleb)
    
modelsPTf = modelsPretrained + "/" + tf_mm
modelsPMask = modelsPretrained + "/" + od_mask
modelsPCeleb = modelsPretrained + "/" + c_class
if not os.path.exists(modelsPTf):
    os.mkdir(modelsPTf)
if not os.path.exists(modelsPMask):
    os.mkdir(modelsPMask)
if not os.path.exists(modelsPCeleb):
    os.mkdir(modelsPCeleb)

In [ ]:
# tests subfolder
testsPath = f'{abspath}/tests/'
if not os.path.exists(testsPath):
    os.mkdir(testsPath)
    
testsVersion = testsPath + "/" + version
if not os.path.exists(testsVersion):
    os.mkdir(testsVersion)
    
testsImages = testsVersion + "/" + "images"
testsVideos = testsVersion + "/" + "videos"
testsOptVideos = testsVersion + "/" + "videos_optimized"
if not os.path.exists(testsImages):
    os.mkdir(testsImages)
if not os.path.exists(testsVideos):
    os.mkdir(testsVideos)
if not os.path.exists(testsOptVideos):
    os.mkdir(testsOptVideos)

In [ ]:
# For downloads
celeb_file = open("./assets/list_promis.txt", "r")
query_array = celeb_file.read().split(',\n')

# For classifcation
celeb_array = sorted(query_array)
celeb_array.append("Unknown")
celeb_nbr = len(celeb_array)

In [ ]:
# Paths for pretrained models
tf_mm_pretrained = f"{modelsPTf}/model.tflite"
obj_det_masks_pretrained = f"{modelsPMask}/"
celeb_classifier_pretrained = f"{modelsPCeleb}/"

In [ ]:
# Mask requirements (Datasets needs to be manually downloaded from kaggle: https://www.kaggle.com/wobotintelligence/face-mask-detection-dataset)
downloaded = False
if(downloaded):
    directory = f"{maskMasks}/Medical mask/Medical mask/Medical Mask/annotations"
    OC_Net = cv2.dnn.readNetFromCaffe(f"{maskCaffe}/architecture.txt", f'{maskCaffe}/weights.caffemodel')
    df = pd.read_csv(f"{maskMasks}/train.csv")
    image_directory = f"{maskMasks}/Medical mask/Medical mask/Medical Mask/images"
    image_test_directory = f"{maskMasks}/Medical mask/Medical mask/Medical Mask/images/"

In [ ]:
# General requirements (change to own trained models)
classifier = keras.models.load_model(celeb_classifier_pretrained)
maskModel = keras.models.load_model(obj_det_masks_pretrained)